In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train_features.csv
/kaggle/input/train_salaries.csv
/kaggle/input/test_features.csv


In [2]:
import pandas as pd
test_features = pd.read_csv("../input/test_features.csv")
train_features = pd.read_csv("../input/train_features.csv")
train_salaries = pd.read_csv("../input/train_salaries.csv")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [4]:
def load_file(file):
    '''loads csv to pd dataframe'''
    return pd.read_csv(file)

def consolidate_data(df1, df2, key=None, left_index=False, right_index=False):
    '''perform inner join to return only records that are present in both dataframes'''
    return pd.merge(left=df1, right=df2, how='inner', on=key, left_index=left_index, right_index=right_index)

def clean_data(raw_df):
    '''remove rows that contain salary <= 0 or duplicate job IDs'''
    clean_df = raw_df.drop_duplicates(subset='jobId')
    clean_df = clean_df[clean_df.salary>8.25]
    return clean_df

def one_hot_encode_feature_df(df, cat_vars=None, num_vars=None):
    '''performs one-hot encoding on all categorical variables and combines result with continous variables'''
    cat_df = pd.get_dummies(df[cat_vars])
    num_df = df[num_vars].apply(pd.to_numeric)
    return pd.concat([cat_df, num_df], axis=1)#,ignore_index=False)

def get_target_df(df, target):
    '''returns target dataframe'''
    return df[target]

# We take 5times cross validation.
def train_model(model, feature_df, target_df, num_procs, mean_mse, cv_std):
    neg_mse = cross_val_score(model, feature_df, target_df, cv=5, n_jobs=num_procs, scoring='neg_mean_squared_error')
    mean_mse[model] = -1.0*np.mean(neg_mse)
    cv_std[model] = np.std(neg_mse)

def print_summary(model, mean_mse, cv_std):
    print('\nModel:\n', model)
    print('Average MSE:\n', mean_mse[model])
    print('Standard deviation during CV:\n', cv_std[model])

def save_results(model, mean_mse, predictions, feature_importances):
    '''saves model, model summary, feature importances, and predictions'''
    with open('model.txt', 'w') as file:
        file.write(str(model))
    feature_importances.to_csv('feature_importances.csv') 
    np.savetxt('predictions.csv', predictions, delimiter=',')

In [5]:
# Load data
feature_df = train_features
target_df = train_salaries
test_df = test_features

#define variables
categorical_vars = ['companyId', 'jobType', 'degree', 'major', 'industry']
numeric_vars = ['yearsExperience', 'milesFromMetropolis']
target_var = 'salary'

#consolidate training data
raw_train_df = consolidate_data(feature_df, target_df, key='jobId')

#clean, shuffle, and reindex training data -- shuffling improves cross-validation accuracy
clean_train_df = shuffle(clean_data(raw_train_df)).reset_index()

#encode categorical data and get final feature dfs
print("Encoding data")
feature_df = one_hot_encode_feature_df(clean_train_df, cat_vars=categorical_vars, num_vars=numeric_vars)
test_df = one_hot_encode_feature_df(test_df, cat_vars=categorical_vars, num_vars=numeric_vars)

#get target df
target_df = get_target_df(clean_train_df, target_var)

Encoding data


In [6]:
#initialize model list and dicts
models = []
mean_mse = {}
cv_std = {}
res = {}

#define number of processes to run in parallel
num_procs = 2

#shared model paramaters
verbose_lvl = 0

# BUILDING NEURAL NETWORK

We tried several combinations and architecture of Multi Layer perceptron model. Here we present the best model which we found.

In [7]:
X_train = np.array(feature_df)
y_train = np.array(target_df)
type(X_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [8]:
print(target_df.shape), print(feature_df.shape)

(999995,)
(999995, 94)


(None, None)

In [9]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense

model1 = Sequential([
    Dense(units = 50, activation = 'relu', input_shape=(94,)), 
    #Dense(units = 1000, activation = 'relu'), 
      # Number of rows are not part of input_shape. This is hidden layer of x neurons.
    Dense(units = 1, activation = 'relu')
    ])
model1.compile(optimizer = 'adam', loss = 'mean_squared_error')
model1.fit(X_train, y_train, epochs = 10)

Using TensorFlow backend.


Epoch 1/40
999995/999995 [==============================] - 125s 125us/step - loss: 472.9989
Epoch 2/40
999995/999995 [==============================] - 128s 128us/step - loss: 360.8550
Epoch 3/40
999995/999995 [==============================] - 124s 124us/step - loss: 359.3317
Epoch 4/40
999995/999995 [==============================] - 122s 122us/step - loss: 358.7237
Epoch 5/40
999995/999995 [==============================] - 124s 124us/step - loss: 358.4102
Epoch 6/40
999995/999995 [==============================] - 124s 124us/step - loss: 357.9478
Epoch 7/40
999995/999995 [==============================] - 123s 123us/step - loss: 357.6407
Epoch 8/40
999995/999995 [==============================] - 124s 124us/step - loss: 357.2639
Epoch 9/40
999995/999995 [==============================] - 124s 124us/step - loss: 357.0462
Epoch 10/40
999995/999995 [==============================] - 124s 124us/step - loss: 356.8835
Epoch 11/40
217568/999995 [=====>........................] - ETA: 1:4

KeyboardInterrupt: 

# Mean Squared Error = 356.8835 (after 10th epoch)

# MSE for other Activation functions
* elu -356.9582
* selu - 356.7538
* softmax - 14738.3727
* softplus - 357.0105
* softsign - 14738.3900
* relu - 356.9876
* sigmoid - 14738
* tanh - 14738
* hard-sigmoid - 14738.3727
* exponential - nan
* linear - 387.0325